In [20]:
# Copyright (c) 2021, Hyunwoong Ko. Modified 2022 by Billy Cao.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
import json
import torch
import numpy as np
from transformers import AutoModelForSeq2SeqLM, BartTokenizer
from flask import Flask, request, jsonify
import onnx
from onnxruntime import InferenceSession


session = InferenceSession("onnx_model/model.onnx", providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
model = onnx.load("onnx_model/model.onnx")
tokenizer = BartTokenizer.from_pretrained('ctrlsum-cnndm')

source_text = "Lenovo Legion 5 Pro Gen 6 laptop. CPU: Ryzen 7 5800H, GPU: RTX 3070 (8 GB VRAM), memory: 32GB, storage: 512GB PCIe NVMe. Contact me at +65 9127 3842 or yeet@somemail.com."
prompt = "The seller's phone number is:"
input_text = f"{prompt} => {source_text}"

inputs = tokenizer(input_text, return_tensors="np")
decoder_inputs = tokenizer(prompt, return_tensors="np")
outputs = session.run(output_names=["last_hidden_state"], input_feed={"input_ids": inputs["input_ids"].astype(np.int64), "attention_mask": inputs["attention_mask"].astype(np.int64), "decoder_input_ids": decoder_inputs['input_ids'].astype(np.int64), "decoder_attention_mask": decoder_inputs["attention_mask"].astype(np.int64)})
